In [1]:
%pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=384089538033564cc08e305d497f626049e6b7f91bd5360b73a84693bbf5115b
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg
from pyspark.sql.functions import current_date, datediff, floor, col, expr, when
from pyspark.sql.functions import col, current_date, datediff, to_date
from pyspark.sql.types import DateType

# Cria a sessão Spark
spark = SparkSession.builder \
    .appName("CSV") \
    .getOrCreate()

In [3]:
# Redefinir nomes de colunas e atribuir o tipo devido a elas

df = spark.read.csv('/content/base_copa2018.csv', header=True, inferSchema=True)

df = df.withColumn('Birth Date', to_date(col('Birth Date'), 'dd.MM.yyyy'))

df = df.withColumn('Height', col('Height').cast('int'))

df = df.withColumnRenamed('Pos.', 'Position')

df = df.withColumnRenamed('#', 'Number')

df = df.withColumn('Weight', col('Weight').cast('int'))

df = df.withColumn('Number', col('Number').cast('int'))

df.show()


+---------+------+--------+------------------+----------+----------+--------------------+------+------+
|     Team|Number|Position| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|
+---------+------+--------+------------------+----------+----------+--------------------+------+------+
|Argentina|     3|      DF|TAGLIAFICO Nicolas|1992-08-31|TAGLIAFICO|      AFC Ajax (NED)|   169|    65|
|Argentina|    22|      MF|    PAVON Cristian|1996-01-21|     PAVÓN|CA Boca Juniors (...|   169|    65|
|Argentina|    15|      MF|    LANZINI Manuel|1993-02-15|   LANZINI|West Ham United F...|   167|    66|
|Argentina|    18|      DF|    SALVIO Eduardo|1990-07-13|    SALVIO|    SL Benfica (POR)|   167|    69|
|Argentina|    10|      FW|      MESSI Lionel|1987-06-24|     MESSI|  FC Barcelona (ESP)|   170|    72|
|Argentina|     4|      DF|  ANSALDI Cristian|1986-09-20|   ANSALDI|     Torino FC (ITA)|   181|    73|
|Argentina|     5|      MF|      BIGLIA Lucas|1986-01-30|    BIG

# i. Qual é a média de altura de cada seleção? Qual é a seleção mais alta e a mais baixa?

In [5]:
# Calcula a altura média de cada equipe
altura_media_df = df.groupBy("Team").agg(avg(df["Height"].cast("float")).alias("AlturaMedia"))

# Encontra a equipe com a maior altura média
maior_altura_media = altura_media_df.orderBy("AlturaMedia", ascending=False).first()

# Encontra a equipe com a menor altura média
menor_altura_media = altura_media_df.orderBy("AlturaMedia").first()

# Exibe a altura média de cada equipe ordenada de forma decrescente
print("Altura média de cada equipe:")
altura_media_df = altura_media_df.orderBy("AlturaMedia", ascending=False)
altura_media_df.show(32)

# Exibe a equipe com a maior altura média
print("\nEquipe com maior altura média:")
print(maior_altura_media["Team"], ":", f"{maior_altura_media['AlturaMedia']:.2f}")

# Exibe a equipe com a menor altura média
print("\nEquipe com menor altura média:")
print(menor_altura_media["Team"], ":", f"{menor_altura_media['AlturaMedia']:.2f}")

Altura média de cada equipe:
+--------------+------------------+
|          Team|       AlturaMedia|
+--------------+------------------+
|        Serbia|186.69565217391303|
|       Denmark| 186.6086956521739|
|       Germany| 185.7826086956522|
|        Sweden| 185.7391304347826|
|       Iceland|185.52173913043478|
|       Belgium|185.34782608695653|
|       Croatia| 185.2608695652174|
|       Nigeria|184.52173913043478|
|       IR Iran|184.47826086956522|
|        Russia| 184.3913043478261|
|       Senegal|183.65217391304347|
|        France|183.30434782608697|
|        Poland|183.17391304347825|
|       Tunisia|183.08695652173913|
|   Switzerland|182.91304347826087|
|       England| 182.7391304347826|
|       Morocco|182.69565217391303|
|        Panama|182.17391304347825|
|Korea Republic| 181.8695652173913|
|       Uruguay|181.04347826086956|
|         Egypt|             181.0|
|     Australia| 180.8695652173913|
|        Brazil| 180.7826086956522|
|      Colombia| 180.7826086956522|

# ii. Qual é a média de peso de cada seleção? Qual é a seleção mais pesada e a mais leve?

In [6]:
# Calcula o peso médio de cada equipe
peso_medio_df = df.groupBy("Team").agg(avg(df["Weight"].cast("float")).alias("PesoMedio"))

# Encontra a equipe com o peso médio mais alto
selecao_mais_pesada = peso_medio_df.orderBy("PesoMedio", ascending=False).first()

# Encontra a equipe com o peso médio mais baixo
selecao_mais_leve = peso_medio_df.orderBy("PesoMedio").first()

# Exibe o peso médio de cada equipe ordenado de forma decrescente
print("Peso médio de cada seleção:")
peso_medio_df = peso_medio_df.orderBy("PesoMedio", ascending=False)
peso_medio_df.show(32)

# Exibe a equipe com o peso médio mais alto
print("\nSeleção com o peso médio mais alto:")
print(selecao_mais_pesada["Team"], ":", f"{selecao_mais_pesada['PesoMedio']:.2f}")

# Exibe a equipe com o peso médio mais baixo
print("\nSeleção com o peso médio mais baixo:")
print(selecao_mais_leve["Team"], ":", f"{selecao_mais_leve['PesoMedio']:.2f}")


Peso médio de cada seleção:
+--------------+-----------------+
|          Team|        PesoMedio|
+--------------+-----------------+
|       Denmark| 82.6086956521739|
|       Iceland|80.73913043478261|
|        Serbia|80.52173913043478|
|       Nigeria|80.47826086956522|
|       Germany|80.04347826086956|
|        France|             80.0|
|        Panama|             80.0|
|   Switzerland|79.91304347826087|
|       Belgium|79.56521739130434|
|       Croatia|79.30434782608695|
|        Sweden|78.82608695652173|
|       England|78.82608695652173|
|         Egypt|78.43478260869566|
|       IR Iran| 78.1304347826087|
|     Australia|77.73913043478261|
|        Russia| 77.6086956521739|
|       Senegal|76.82608695652173|
|        Brazil|76.56521739130434|
|        Poland|76.47826086956522|
|      Colombia|76.17391304347827|
|          Peru| 75.8695652173913|
|     Argentina|75.56521739130434|
|       Tunisia|             75.0|
|         Spain|74.73913043478261|
|       Morocco|74.65217391

# iii. Qual é a media de idade de clube?

In [7]:
# Calcula a idade atual de cada jogador e adiciona como uma nova coluna 'idade_atual' no DataFrame df
df = df.withColumn('idade_atual', (datediff(current_date(), 'Birth Date') / 365).cast('int'))

# Calcula a média de idade para cada equipe no DataFrame df e armazena o resultado em media_idade_df
media_idade_df = df.groupBy("Club").agg(avg(df["idade_atual"]).alias("MediaIdade"))

# Exibe a média de idade por clube ordenada de forma decrescente pela coluna "MediaIdade"
print("Média de idade por clube:")
media_idade_df_show = media_idade_df.orderBy("MediaIdade", ascending=False)
media_idade_df_show.show(310)

Média de idade por clube:
+--------------------+------------------+
|                Club|        MediaIdade|
+--------------------+------------------+
| Al Taawoun FC (KSA)|              51.0|
|      CF Atlas (MEX)|              45.0|
| CSD Municipal (GUA)|              43.0|
|     Cruz Azul (MEX)|              43.0|
|    Goztepe SK (TUR)|              42.0|
|    Rangers FC (SCO)|              42.0|
|    Sport Boys (PER)|              42.0|
|Dinamo Bucharest ...|              42.0|
|CR Vasco da Gama ...|              41.0|
|   Karabukspor (TUR)|              41.0|
|       FC Metz (FRA)|              41.0|
|Deportivo Toluca ...|              41.0|
|   Aberdeen FC (SCO)|              41.0|
|   Millwall FC (ENG)|              40.0|
|Atlético Bucarama...|              40.0|
|Club Brugge KV (BEL)|              40.0|
|Atletico Junior (...|              40.0|
|    AEK Athens (GRE)|              40.0|
|Hebei China Fortu...|              40.0|
|Rionegro Águilas ...|              40.0|
|   FK P

# iv. Qual clube tem o jogador com IMC (Índice de massa corporal) mais alto da competição?

In [8]:
# Calcula o Índice de Massa Corporal (IMC) para cada jogador e adiciona como uma nova coluna 'IMC' no DataFrame df
df = df.withColumn('IMC', expr('Weight / (Height / 100 * Height / 100)'))

# Encontra o jogador com o maior IMC ao ordenar o DataFrame df de forma decrescente pela coluna 'IMC' e selecionar o primeiro registro
max_imc_jogador = df.orderBy(col('IMC').desc()).select('Club', 'FIFA Popular Name', 'IMC').first()

# Verifica se foi encontrado algum jogador com IMC
if max_imc_jogador:
    # Extrai informações do jogador com maior IMC
    time = max_imc_jogador['Club']
    jogador = max_imc_jogador['FIFA Popular Name']
    max_IMC = max_imc_jogador['IMC']
    # Imprime mensagem indicando o jogador com o maior IMC encontrado, formatando o IMC com duas casas decimais
    print(f"O jogador com o maior IMC é '{jogador}' do time '{time}' com IMC de {max_IMC:.2f}.")
else:
    # Caso nenhum jogador com IMC seja encontrado, imprime mensagem indicando que não foi possível determinar
    print("Não foi possível determinar o jogador com maior IMC.")


O jogador com o maior IMC é 'ABDULLAH ALMUAIOUF' do time 'Al Hilal SFC (KSA)' com IMC de 30.93.


# v. Qual clube tem o jogador com IMC (Índice de massa corporal) mais baixo da competição?

In [9]:
# Encontra o jogador com o menor IMC ao ordenar o DataFrame df de forma crescente pela coluna 'IMC' e selecionar o primeiro registro
min_imc_jogador = df.orderBy(col('IMC')).select('Club', 'FIFA Popular Name', 'IMC').first()

# Verifica se foi encontrado algum jogador com IMC
if min_imc_jogador:
    # Extrai informações do jogador com menor IMC
    time = min_imc_jogador['Club']
    jogador = min_imc_jogador['FIFA Popular Name']
    min_IMC = min_imc_jogador['IMC']
    # Imprime mensagem indicando o jogador com o menor IMC encontrado, formatando o IMC com duas casas decimais
    print(f"O jogador com o menor IMC é '{jogador}' do time '{time}' com IMC de {min_IMC:.2f}.")
else:
    # Caso nenhum jogador com IMC seja encontrado, imprime mensagem indicando que não foi possível determinar
    print("Não foi possível determinar o jogador com menor IMC.")

O jogador com o menor IMC é 'BRUNO FERNANDES' do time 'Sporting CP (POR)' com IMC de 19.11.


# vi. Qual é o jogador mais velho da competição?

In [10]:
# Encontra o jogador mais velho ao ordenar o DataFrame df de forma decrescente pela coluna 'idade_atual' e selecionar o primeiro registro
jogador_mais_velho = df.orderBy(col('idade_atual').desc()).select('FIFA Popular Name', 'idade_atual').first()

# Verifica se foi encontrado algum jogador mais velho
if jogador_mais_velho:
    # Extrai informações do jogador mais velho
    jogador = jogador_mais_velho['FIFA Popular Name']
    idade = jogador_mais_velho['idade_atual']
    # Imprime mensagem indicando o jogador mais velho encontrado
    print(f"O jogador mais velho da competição é '{jogador}' com {idade} anos de idade.")
else:
    # Caso nenhum jogador mais velho seja encontrado, imprime mensagem indicando que não foi possível determinar
    print("Não foi possível determinar o jogador mais velho.")

O jogador mais velho da competição é 'ESSAM EL HADARY' com 51 anos de idade.


# vii. Qual é o jogador mais novo da competição?

In [11]:
# Encontra o jogador mais novo ao ordenar o DataFrame df de forma crescente pela coluna 'idade_atual' e selecionar o primeiro registro
jogador_mais_novo = df.orderBy(col('idade_atual')).select('FIFA Popular Name', 'idade_atual').first()

# Verifica se foi encontrado algum jogador mais novo
if jogador_mais_novo:
    # Extrai informações do jogador mais novo
    jogador = jogador_mais_novo['FIFA Popular Name']
    idade = jogador_mais_novo['idade_atual']
    # Imprime mensagem indicando o jogador mais novo encontrado
    print(f"O jogador mais novo da competição é '{jogador}' com {idade} anos de idade.")
else:
    # Caso nenhum jogador mais novo seja encontrado, imprime mensagem indicando que não foi possível determinar
    print("Não foi possível determinar o jogador mais novo.")

O jogador mais novo da competição é 'ARZANI Daniel' com 25 anos de idade.


# viii. Qual é o clube com mais jogadores na competição?

In [12]:
# Conta o número de jogadores por clube no DataFrame df
jogadores_por_club = df.groupBy('Club').count()

# Encontra o clube com mais jogadores ao ordenar o DataFrame jogadores_por_club de forma decrescente pelo número de jogadores ('count') e selecionar o primeiro registro
club_com_mais_jogadores = jogadores_por_club.orderBy(col('count').desc()).first()

# Verifica se foi encontrado algum clube com mais jogadores
if club_com_mais_jogadores:
    # Extrai informações do clube com mais jogadores
    clube = club_com_mais_jogadores['Club']
    num_jogadores = club_com_mais_jogadores['count']
    # Imprime mensagem indicando o clube com mais jogadores encontrado
    print(f"O clube com mais jogadores na competição é '{clube}' com {num_jogadores} jogadores.")
else:
    # Caso nenhum clube com mais jogadores seja encontrado, imprime mensagem indicando que não foi possível determinar
    print("Não foi possível determinar o clube com mais jogadores.")

O clube com mais jogadores na competição é 'Manchester City FC (ENG)' com 16 jogadores.


# ix. Monte um time com os jogadores mais altos da competição, respeitando a quantidade de jogadores por posição (1 goleiro, 4 defensores, 4 meio-campistas e 2 atacantes).

In [13]:
# Ordena o DataFrame df de forma decrescente pela coluna 'Height' (altura dos jogadores)
df = df.orderBy(col('Height').desc())

# Filtra e seleciona o goleiro com a maior altura, limitando a 1 registro
goleiro = df.filter(col('Position') == 'GK').limit(1)

# Filtra e seleciona os 4 defensores com as maiores alturas
defensores = df.filter(col('Position') == 'DF').limit(4)

# Filtra e seleciona os 4 meio-campistas com as maiores alturas
meio_campistas = df.filter(col('Position') == 'MF').limit(4)

# Filtra e seleciona os 2 atacantes com as maiores alturas
atacantes = df.filter(col('Position') == 'FW').limit(2)

# Une os DataFrames dos diferentes tipos de jogadores (goleiro, defensores, meio-campistas e atacantes) em um único DataFrame 'team'
team = goleiro.union(defensores).union(meio_campistas).union(atacantes)

# Imprime uma mensagem indicando o time formado com os jogadores mais altos da competição
print("Time formado com os jogadores mais altos da competição:")
# Exibe o DataFrame 'team' sem truncamento para mostrar todas as colunas
team.show(truncate=False)

Time formado com os jogadores mais altos da competição:
+--------------+------+--------+------------------+----------+-----------+----------------------------------+------+------+-----------+------------------+
|Team          |Number|Position|FIFA Popular Name |Birth Date|Shirt Name |Club                              |Height|Weight|idade_atual|IMC               |
+--------------+------+--------+------------------+----------+-----------+----------------------------------+------+------+-----------+------------------+
|Croatia       |12    |GK      |KALINIC Lovre     |1990-04-03|L. KALINIĆ |KAA Gent (BEL)                    |201   |96    |34         |23.761788074552616|
|Denmark       |3     |DF      |VESTERGAARD Jannik|1992-08-03|VESTERGAARD|VfL Borussia Mönchengladbach (GER)|200   |98    |31         |24.5              |
|Argentina     |6     |DF      |FAZIO Federico    |1987-03-17|FAZIO      |AS Roma (ITA)                     |199   |85    |37         |21.464104441807027|
|Costa Rica   

# x. Monte um time com os jogadores mais pesados da competição, respeitando a quantidade de jogadores por posição (1 goleiro, 4 defensores, 4 meio-campistas e 2 atacantes).

In [14]:
# Ordena o DataFrame df de forma decrescente pela coluna 'Weight' (peso dos jogadores)
df = df.orderBy(col('Weight').desc())

# Filtra e seleciona o goleiro mais pesado, limitando a 1 registro
goleiro = df.filter(col('Position') == 'GK').limit(1)

# Filtra e seleciona os 4 defensores mais pesados
defensores = df.filter(col('Position') == 'DF').limit(4)

# Filtra e seleciona os 4 meio-campistas mais pesados
meio_campistas = df.filter(col('Position') == 'MF').limit(4)

# Filtra e seleciona os 2 atacantes mais pesados
atacantes = df.filter(col('Position') == 'FW').limit(2)

# Une os DataFrames dos diferentes tipos de jogadores (goleiro, defensores, meio-campistas e atacantes) em um único DataFrame 'team'
team = goleiro.union(defensores).union(meio_campistas).union(atacantes)

# Imprime uma mensagem indicando o time formado com os jogadores mais pesados da competição
print("Time formado com os jogadores mais pesados da competição:")
# Exibe o DataFrame 'team' sem truncamento para mostrar todas as colunas
team.show(truncate=False)

Time formado com os jogadores mais pesados da competição:
+--------------+------+--------+------------------+----------+-----------+----------------------------------+------+------+-----------+------------------+
|Team          |Number|Position|FIFA Popular Name |Birth Date|Shirt Name |Club                              |Height|Weight|idade_atual|IMC               |
+--------------+------+--------+------------------+----------+-----------+----------------------------------+------+------+-----------+------------------+
|Saudi Arabia  |1     |GK      |ABDULLAH ALMUAIOUF|1987-01-23|ABDULLAH   |Al Hilal SFC (KSA)                |178   |98    |37         |30.930438076000502|
|Panama        |5     |DF      |TORRES Roman      |1986-03-20|R. TORRES  |Seattle Sounders FC (USA)         |188   |99    |38         |28.010411951109102|
|Denmark       |3     |DF      |VESTERGAARD Jannik|1992-08-03|VESTERGAARD|VfL Borussia Mönchengladbach (GER)|200   |98    |31         |24.5              |
|England    